#                   Sentiment Analisys
#                   Ilan Moshe (024176398)
                

##In the Product_handling file , we load a dataset of customers product review , Run customers review on the model for sentiment analysis ,  And then check all review results per product to see what product is good and what product is problematic in customers point of view .
## In the product file test file , There is no 0/1 indication for sentiment analysis , but in addition to the verbal customer review , There is an rating (1-5) given by the customer that can be used as another indication if the sentinment analysis prediction is corrent ot not.
## I comare the results from the model based on the customer sentiment to the start rating made by the customers (rated 1-5 stars) for additional comparission criterion.
## Finally I set a threshold for excelent , good , bad products  based on the customer verbal reviews and the sentiment analysis model results

### Package installtion

In [1]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## Import Packaged

In [42]:
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import json
import shutil
import re as re
import torch
import plotly.express as px
import numpy as np
from sklearn.metrics import accuracy_score
from pprint import pprint
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## Dowload data from GIT and google drive


*   Download the trained model for sentiment classfication
*   Download the CSV containing the customer reviews about the products



In [8]:
# Download the Model from GitHub
!git clone --filter=blob:none --sparse https://github.com/ilanmoshe52/NLP/
!unzip /content/NLP/fine_tuned_model.zip -d /content/fine_tuned_model
!rm -r NLP

Cloning into 'NLP'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 5), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (15/15), 6.46 KiB | 6.46 MiB/s, done.
Resolving deltas: 100% (5/5), done.
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (8/8), 24.76 MiB | 18.58 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (8/8), done.
Archive:  /content/NLP/fine_tuned_model.zip
  inflating: /content/fine_tuned_model/id2label.json  
  inflating: /content/fine_tuned_model/README.md  
  inflating: /content/fine_tuned_model/label2id.json  
  inflating: /content/fine_tuned_model/tokenizer.json  
  inflating: /content/fine_tuned_model/vocab.txt  
  inflating: /content/fine_tuned_model/adapter_con

In [14]:
# Download the Product customer reviews file from GitHub
!wget -O filtered_reviews.csv  "https://github.com/ilanmoshe52/NLP/blob/b96c3d75fb491ce0edd9fdd26c6655ebc337c6d1/filtered_reviews.csv"

--2024-07-23 08:36:33--  https://github.com/ilanmoshe52/NLP/blob/b96c3d75fb491ce0edd9fdd26c6655ebc337c6d1/filtered_reviews.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘filtered_reviews.csv’

filtered_reviews.cs     [ <=>                ] 547.96K  --.-KB/s    in 0.1s    

2024-07-23 08:36:34 (4.37 MB/s) - ‘filtered_reviews.csv’ saved [561113]



## Load the model

In [10]:
# how to load peft model from hub for inference
model_id = extract_path
config = PeftConfig.from_pretrained(model_id)

In [11]:
# Load id2label and label2id mappings
with open('/content/fine_tuned_model/id2label.json', 'r') as f:
    id2label = json.load(f)
with open('/content/fine_tuned_model/label2id.json', 'r') as f:
    label2id = json.load(f)

In [12]:
inference_model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path, num_labels=2, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(inference_model, model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Handle the product review data

In [49]:
import pandas as pd

encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

# Load the CSV file
for encoding in encodings:
    try:
        df = pd.read_csv('/content/filtered_reviews.csv', encoding=encoding ,  on_bad_lines='warn')
        print(f"Successfully read the file with {encoding} encoding.")
        break
    except UnicodeDecodeError as e:
        print(f"Failed to read with {encoding} encoding. Error: {e}")

# Display the first few rows of the DataFrame
df.head()


Successfully read the file with utf-8 encoding.


,ProductId,Text,Score
0,B004PEGIJY,"At the time of this review, Zevia Grape is not...",5
1,B004PEGIJY,I want to say up front that I hate the taste o...,5
2,B004PEGIJY,If I was a fan of grape soda then I would cert...,4
3,B004PEGIJY,CAUTION...DO NOT use ice cubes in any Zevia so...,4
4,B004PEGIJY,"I bought these for my husband at WF, because h...",5


### Clean the data from special characters

In [19]:
# Function to remove special tags from the dataset
def clean_text(text):
    import re
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    return text

In [20]:
df['Text']=df['Text'].apply(lambda cw : clean_text(cw))
df.head()

,ProductId,Text,Score
0,B004PEGIJY,at the time of this review zevia grape is not ...,5
1,B004PEGIJY,i want to say up front that i hate the taste o...,5
2,B004PEGIJY,if i was a fan of grape soda then i would cert...,4
3,B004PEGIJY,cautiondo not use ice cubes in any zevia sodai...,4
4,B004PEGIJY,i bought these for my husband at wf because he...,5


## Group all customers reivews per product

In [106]:
# Group by 'ProductId'
grouped = df.groupby('ProductId')

# Create a dictionary with ProductId as key and list of lists of 'Text' as values
product_reviews_dict = {
    product_id: {
        'Texts': group['Text'].tolist(),
        'Scores': group['Score'].tolist()
    }
    for product_id, group in grouped
}


### Calculate the average score by product ( This will be used as reference )

In [22]:
# Calculate the average score by product
avg_scores = df.groupby('ProductId')['Score'].mean().reset_index()
avg_scores['Score'] = avg_scores['Score'] * 20

### General product customer satisfaction by user stars review

In [108]:
score_counts = df['Score'].value_counts().reset_index()
score_counts.columns = ['Score', 'Count' ]

# Create the pie chart
fig = px.pie(score_counts, values='Count', names='Score', title='Percentage of Each Customer Score Rating')
fig.show()

### Run Customers reviews of each item on the model

In [24]:
# copy the original dictionary
updated_dict = {
    key: (value if isinstance(value, dict) else {'OriginalValue': value, 'predictions': 0})
    for key, value in product_reviews_dict.items()
}

# Add 'MeanValue' to existing dictionaries
for key, value in product_reviews_dict.items():
    if isinstance(value, dict) and 'predictions' not in value:
        value['predictions'] = 0

for key, value in updated_dict.items():

  texts = product_reviews_dict[key]['Texts']

  # Tokenize the input texts
  inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

  # Run the model
  with torch.no_grad():
      outputs = model(**inputs)

  # Get the logits and convert to probabilities (optional)
  logits = outputs.logits
  probs = torch.nn.functional.softmax(logits, dim=-1)

  # Get the predicted class indices
  predictions = torch.argmax(logits, dim=-1)

  # Update the 'predictions' field in the dictionary
  updated_dict[key]['predictions'] = predictions



### Generate Socres vectors based on **accuaracy score** of model's results for all Customers reviews

In [104]:
# generate a reulsts dict
final_dict = {}
total_ground = 0
total_pred = 0
total_items = 0
for key, value in updated_dict.items():

  scores = product_reviews_dict[key]['Scores']
  y_pred = product_reviews_dict[key]['predictions']

  # Convert to a list of 0s and 1s
  y_true = [round(num / 5) for num in scores]

  total_ground +=  np.sum(y_true)
  total_pred += np.sum(list(y_pred))
  total_items += np.sum(len(y_true))

  acc = accuracy_score(y_true, y_pred)
  calc_acc =  (np.sum(list(y_pred))/len(y_pred))*100

  final_dict[key] = np.round(acc*100)

  # Print the converted list
  print("Summary for product " + key + ': ' +  ' Product Accuracy: ' + str(np.ceil(acc*100)) + ' (' + str(calc_acc) + ')')

# Convert the dictionary to a pandas DataFrame
df_final = pd.DataFrame(list(final_dict.items()), columns=['Item', 'Score'])



Summary for product B0001BH5YM:  Product Accuracy: 90.0 (89.47368421052632)
Summary for product B0002ARRCC:  Product Accuracy: 69.0 (42.10526315789473)
Summary for product B0005XO2ME:  Product Accuracy: 37.0 (73.68421052631578)
Summary for product B000EGSUMU:  Product Accuracy: 69.0 (68.42105263157895)
Summary for product B000EVIDK4:  Product Accuracy: 100.0 (94.73684210526315)
Summary for product B000FIULB0:  Product Accuracy: 74.0 (78.94736842105263)
Summary for product B000GZUAFE:  Product Accuracy: 90.0 (84.21052631578947)
Summary for product B000LKTXIO:  Product Accuracy: 90.0 (89.47368421052632)
Summary for product B000LKYQJ0:  Product Accuracy: 90.0 (94.73684210526315)
Summary for product B000N8QJUE:  Product Accuracy: 90.0 (26.31578947368421)
Summary for product B000W5U5H6:  Product Accuracy: 69.0 (68.42105263157895)
Summary for product B000WL58J0:  Product Accuracy: 79.0 (57.89473684210527)
Summary for product B0010Y6QBC:  Product Accuracy: 74.0 (78.94736842105263)
Summary for

In [105]:
print('Total true prediction related to the customer socres is : ' , str(total_pred/total_ground))

Total true prediction related to the customer socres is :  0.8893617021276595


In [72]:
#avg_scores['Score'] = avg_scores['Score'] * 20

## **Prepare Plots:**

###    - A graph of accuacy scores of the Predictions of the model
###    - A graph based on customers stars rating of each product



In [89]:
fig1 = px.bar(df_final, x='Item', y='Score', title='Final')
fig2 = px.bar(avg_scores, x='ProductId', y='Score', title='Average Score by Product')

In [109]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Scores Based on Customer Sentiment Analisys', 'Scores Based on Star Rating Customer Avarage Scores'))

# Add bar charts to subplots
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)

# Update layout
fig.update_layout(title_text='Compare between Model Predictions and customers Rating  ')

# Show plot
fig.show()

# We define the following scale for product status :
  ### - 0 - 50%    - Products that are problematic and discontinued
  ### - 51% - 80%  - Products that need to be upgraded
  ### - 81% - 100% - Excelent Products


In [46]:
# Initialize the dictionary with empty lists for each category
score_dict = {
    'Problematic': [],
    'Good': [],
    'Excellent': []
}

# Iterate through the DataFrame and categorize each score
for _, row in df_final.iterrows():
    product_id = row['Item']
    score = row['Score']

    #print(product_id , score)
    if 0 <= score <= 50:
        score_dict['Problematic'].append(product_id)
    elif 51 <= score <= 80:
        score_dict['Good'].append(product_id)
    elif 81 <= score <= 100:
        score_dict['Excellent'].append(product_id)

print('List of products rated by Sentiment analisys: \n')
pprint(score_dict)

List of products rated by Sentiment analisys: 

{'Excellent': ['B0001BH5YM',
               'B000EVIDK4',
               'B000GZUAFE',
               'B000LKTXIO',
               'B000LKYQJ0',
               'B000N8QJUE',
               'B0013M07PW',
               'B001EQ5F9K',
               'B001HTE5L8',
               'B002ATCFX4',
               'B002AUF15C',
               'B003UEKFS0',
               'B004PEGIJY'],
 'Good': ['B0002ARRCC',
          'B000EGSUMU',
          'B000FIULB0',
          'B000W5U5H6',
          'B000WL58J0',
          'B0010Y6QBC',
          'B0012W1TQE',
          'B0015MLVAA',
          'B001BCVEBU',
          'B001CCXVPQ',
          'B001EQ4E3I',
          'B001SER0L4',
          'B003TN6FNG',
          'B0042YU5EQ',
          'B0098C6AKK'],
 'Problematic': ['B0005XO2ME', 'B001BKR5L0']}
